In [19]:
# In this cell, I query database using sqlite, which returns get back a list of tuples.

import sqlite3
import os

# This connector will search for the file named 'stcv.sqlite' in the same directory as this notebook.
# The dot indicates the relative path connected to the same folder
conn = sqlite3.connect(os.path.join('.','stcv.sqlite'))

# Create cursor object
cur = conn.cursor()

# Use the given query
query = """
SELECT DISTINCT
    title.cloi as identifier,
    COUNT(title.cloi) as id_count,
    author_vw as author_standardized,
    author_zvwr as author_original,
    author_zbd as author_dates,
    corporateauthor_nm as corporateauthor_standardized,
    corporateauthor_zvwr as corporateauthor_original,
    title_ti as title_title,
    title_lg as title_language,
    collation_fm as format,
    collation_ka as quires,
    collation_pg as pages,
    edition_ed as edition_info,
    impressum_ju1sv as year1,
    impressum_ju1ty as year1_type,
    impressum_ju2sv as year2,
    impressum_ju2ty as year2_type,
    impressum_pl as place,
    impressum_ug as printer,
    language_lg as language_info,
    number_nr as fingerprint
    FROM title
LEFT JOIN author on author.cloi = title.cloi
LEFT JOIN collation on collation.cloi = title.cloi
LEFT JOIN corporateauthor on corporateauthor.cloi = title.cloi
LEFT JOIN edition on edition.cloi = title.cloi
LEFT JOIN impressum on impressum.cloi = title.cloi
LEFT JOIN language on language.cloi = title.cloi
LEFT JOIN number on number.cloi = title.cloi
GROUP BY identifier
HAVING ID_COUNT=2
LIMIT 100
"""

# I execute the query
cur.execute(query)

# I iterate over every match and create a list consisting of all resulting rows
data = [row for row in cur.fetchall()]
for row in data:
    print(row)

# Close the database connection
conn.close()

('c:stcv:12854444', 2, 'Ernst, Simon-Pierre', 'S.P. Ernst', '1744-1817', None, None, 'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures', 'fre', '', 'π<sup>2</sup> a<sup>8</sup> b<sup>4</sup> A-H<sup>8</sup> I<sup>6</sup> (- I6) K<sup>2</sup>', 'xxviij, 142 p.', None, '1797', 'YYYY', '', 'YYYY', 's.l.', 's.n.', 'fre', '179708 - # a1 a é : # a2 b2 e, - # b1 A t : # b2 K $a')
('c:stcv:12854501', 2, 'Ernst, Simon-Pierre', 'S.P. Ernst', '1744-1817', None, None, 'Observations sur la déclaration exigée des ministres des cultes, en vertu de la loi du 7 vendémiaire, an 4', 'fre', '', 'π1 A<sup>8</sup> B-D<sup>4</sup> E<sup>4</sup> (- E1)', '[1], [1 blank], 45, [1 blank] p.', None, '1797', 'YYYY', '', 'YYYY', 's.l.', 's.n.', 'fre', '179708 - # b1 A2 vais : # b2 E m')
('c:stcv:12854549', 2, 'Gisbert, Blaise', 'Gisbert', '1657-1731', None, None, "l'Eloquence ch

In [2]:
# The second cell matches all 100 queries to the HPB database using SRU-CQL. The result is the xml in string format.

import urllib.parse
import urllib.request
import urllib.error
import time

# This is the base of the link:
HPB = 'https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query='

# I define how I want to clean the input search:
def clean(string: str) -> str:
    string = string.strip()
    string = string.replace('.', '') # I delete all full stops, as including those yielded wrong results.
    string = string.casefold() 
    string = urllib.parse.quote(string)
    return string


# I define on what metadata I want to match. I decided to match on the title:
def search_for(book: list) -> str:
    search_for = clean(book[7]) # The seventh element in the tuple contains the title
    return search_for

# I query the HPB database:
def query_HPB(search: str) -> bytes:
    time.sleep(2)

    url = HPB + search + '&startRecord=1&maximumRecords=100&recordSchema=marcxml'
    try:
        with urllib.request.urlopen(url) as query:
            return query.read()
    except urllib.error.HTTPError as HTTPerr:
        exit(HTTPerr.code)
    except urllib.error.URLError as URLerr:
        exit(URLerr)

search = search_for(data[0]) # I test this out on the first title in the list from stcv.
print(str(query_HPB(search)))


b'<?xml version="1.0" encoding="UTF-8"?>\n<zs:searchRetrieveResponse xmlns:zs="http://docs.oasis-open.org/ns/search-ws/sruResponse"><zs:numberOfRecords>1</zs:numberOfRecords><zs:records><zs:record><zs:recordSchema>marcxml</zs:recordSchema><zs:recordXMLEscaping>xml</zs:recordXMLEscaping><zs:recordData><record xmlns="http://www.loc.gov/MARC21/slim">\n  <leader>xxxxxnam a22yyyyy c 4500</leader>\n  <controlfield tag="001">15364091X</controlfield>\n  <controlfield tag="003">DE-601</controlfield>\n  <controlfield tag="005">20210820093419.0</controlfield>\n  <controlfield tag="008">090409s1797                  000 0 fre d</controlfield>\n  <datafield tag="035" ind1=" " ind2=" ">\n    <subfield code="a">BE-AnVE.c:stcv:12854444</subfield>\n  </datafield>\n  <datafield tag="040" ind1=" " ind2=" ">\n    <subfield code="b">ger</subfield>\n    <subfield code="c">GBVCP</subfield>\n  </datafield>\n  <datafield tag="041" ind1="0" ind2=" ">\n    <subfield code="a">fre</subfield>\n  </datafield>\n  <dat

In [3]:
# Now, I create a dictionary which includes all relevant metadata of the titles from the HPB database.

import lxml.etree
identifier = search_for(data[0]) # I use the first title to try this part out.
search1 = HPB + identifier

# This is the namespace of the "actual" records:
HPB_XML_NS = r'{http://www.loc.gov/MARC21/slim}'
# This is the namespace of the meta-meta information:
HPB_XML_NS1 = "http://docs.oasis-open.org/ns/search-ws/sruResponse"



def parse(oai: bytes) -> list:
    root = lxml.etree.fromstring(oai)
    records = []
    for record in root.iter(f"{HPB_XML_NS}record"):
        metadata = {}
        index = 0
        for datafield in record.iter(f"{HPB_XML_NS}datafield"):
            for key, value in datafield.items():
                if key == "tag":
                    for subfield in datafield:
                        index += 1
                        for _, code in subfield.items():
                            metadata[index] = {"tag": value,
                                               "code": code,
                                               "code2": subfield.text}
        records.append(metadata)
    return records

parsed = parse(query_HPB(search1))
sqlbook = data[0]
hpbbook = parsed[0]
print(parsed[0])


{1: {'tag': '035', 'code': 'a', 'code2': 'BE-AnVE.c:stcv:12854444'}, 2: {'tag': '040', 'code': 'b', 'code2': 'ger'}, 3: {'tag': '040', 'code': 'c', 'code2': 'GBVCP'}, 4: {'tag': '041', 'code': 'a', 'code2': 'fre'}, 5: {'tag': '044', 'code': 'c', 'code2': 'ZZ'}, 6: {'tag': '100', 'code': 'a', 'code2': 'Ernst, Simon-Pierre'}, 7: {'tag': '100', 'code': 'd', 'code2': '1744-1817'}, 8: {'tag': '100', 'code': 'e', 'code2': '*aut'}, 9: {'tag': '245', 'code': 'a', 'code2': 'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures'}, 10: {'tag': '245', 'code': 'c', 'code2': 'Simon-Pierre Ernst [aut]'}, 11: {'tag': '264', 'code': 'a', 'code2': '[s.l.]'}, 12: {'tag': '264', 'code': 'b', 'code2': '[s.n.]'}, 13: {'tag': '264', 'code': 'c', 'code2': '1797'}, 14: {'tag': '300', 'code': 'a', 'code2': 'xxviij, 142 p. π<sup>2</sup> a<sup>8</sup> b<sup>4</sup> A-H<sup>8</sup> 

In [4]:
# There is also a possibility that the author's name is formatted differently than in the HPB database. For that
# reason, I create a function that switches the first and last name:

def reverse_name(hpbbook):
    for item in hpbbook:
        if hpbbook[item]['tag'] == '100' and hpbbook[item]['code'] == 'a':
            author = hpbbook[item]['code2']
            if ',' in author:
                author_firstname_surname = author.split(',')[1] + ' ' + author.split(',')[0]
                return author_firstname_surname
            else:
                return author
print(reverse_name(hpbbook))


 Simon-Pierre Ernst


In [5]:
# I make a definition that checks whether the author's name from the stcv database is the same in the hpb database:


def author_name_match(sqlbook: list, hpbbook: dict):
    for item in hpbbook:
        if hpbbook[item]['tag'] == '100' and hpbbook[item]['code'] == 'a':
            author = hpbbook[item]['code2']
            if author:
                if author == sqlbook[2] or reverse_name(hpbbook) == sqlbook[2]:
                    return True
                else:
                    return False
            else:
                return False
        
author_name_match(sqlbook, parsed[0]) # I try it out with the first work.

True

In [6]:
# I make a definition that checks whether the date from the stcv database is the same in the hpb database:

def date_match(sqlbook: list, hpbbook: dict):
    for item in hpbbook:
        if hpbbook[item]['tag'] == '100' and hpbbook[item]['code'] == 'd':
            date = hpbbook[item]['code2']
            if date == sqlbook[4]:
                return True
            else:
                return False

date_match(sqlbook, parsed[0]) # I try it out with the first work.

True

In [7]:
# This is the eventual desired outcome.

def result_HPB(hpbbook) -> str:
    for item in hpbbook:
        if hpbbook[item]['tag'] == '035' and hpbbook[item]['code'] == 'a':
            result = hpbbook[item]['code2']
            return result

result_HPB(parsed[0])

'BE-AnVE.c:stcv:12854444'

In [8]:
# This is to select the best match to be returned: in the event of multiple matches, I only want the book that 
# matches the author and date both.

def find_best_match(sqlbook: list, hpbbooks: list) -> dict:
    best_match = hpbbooks[0] # I stack the first hit as temporary best match.
    author_matched = False
    for hpbbook in hpbbooks:
        if author_name_match(sqlbook, hpbbook) and not author_matched: # if this is the first time the author matches:
            best_match = hpbbook
            author_matched = True
        if author_name_match(sqlbook, hpbbook) and date_match(sqlbook, hpbbook): 
            # This is the perfect match. If these criteria fit, end the loop and return this result.
            return hpbbook
        else:
            # If the date and author do not match:
            #print(f"{result_HPB(hpbbook)} \t There is no perfect match, but this one comes close.")
            return hpbbook
            #return f"{hpbbook}\t There is no perfect match, but this one comes close."
    return best_match
find_best_match(sqlbook, parsed)


{1: {'tag': '035', 'code': 'a', 'code2': 'BE-AnVE.c:stcv:12854444'},
 2: {'tag': '040', 'code': 'b', 'code2': 'ger'},
 3: {'tag': '040', 'code': 'c', 'code2': 'GBVCP'},
 4: {'tag': '041', 'code': 'a', 'code2': 'fre'},
 5: {'tag': '044', 'code': 'c', 'code2': 'ZZ'},
 6: {'tag': '100', 'code': 'a', 'code2': 'Ernst, Simon-Pierre'},
 7: {'tag': '100', 'code': 'd', 'code2': '1744-1817'},
 8: {'tag': '100', 'code': 'e', 'code2': '*aut'},
 9: {'tag': '245',
  'code': 'a',
  'code2': 'Apologie des ministres du culte, qui ont prêté la déclaration exigée par la loi du 7 vendém. an 4. contre les critiques de mm. Dédoyar & Vanhoren, les Motifs de Malines & autres brochures'},
 10: {'tag': '245', 'code': 'c', 'code2': 'Simon-Pierre Ernst [aut]'},
 11: {'tag': '264', 'code': 'a', 'code2': '[s.l.]'},
 12: {'tag': '264', 'code': 'b', 'code2': '[s.n.]'},
 13: {'tag': '264', 'code': 'c', 'code2': '1797'},
 14: {'tag': '300',
  'code': 'a',
  'code2': 'xxviij, 142 p. π<sup>2</sup> a<sup>8</sup> b<sup>4</

In [9]:
# This function returns the eventual outcome: the identifier of the STCV book, 
# and the identifier of the best HPB match.

def final(sqlbook: list, hpbbook: dict) -> str:
    display = sqlbook[0]+ ' === ' + result_HPB(hpbbook)
    
    if not author_name_match(sqlbook, hpbbook) and not date_match(sqlbook, hpbbook):
        display += '\tBut author and date didn\'t match' 
    elif not author_name_match(sqlbook, hpbbook):
        display += '\tBut author didn\'t match'
    elif not date_match(sqlbook, hpbbook):
        display += '\tBut date didn\'t match'
    else:
        display += '\tPotential match'
    return display

print(final(sqlbook, parsed[0])) # Again, I try this out on the first work.

c:stcv:12854444 === BE-AnVE.c:stcv:12854444	Potential match


In [18]:
def find_hpb_match(sqlbook: list) -> str:
    search = search_for(sqlbook)
    identifier = search_for(book)
    search_url = HPB + identifier
    parsed = parse(query_HPB(search_url))
    best_match = find_best_match(sqlbook, parsed)
    return(final(sqlbook, best_match))
    

ind = 0
for book in data:
    ind +=1
    print(str(ind)+ '. ' + find_hpb_match(book))    

1. c:stcv:12854444 === BE-AnVE.c:stcv:12854444	Potential match
2. c:stcv:12854501 === BE-AnVE.c:stcv:12913022	But author and date didn't match
3. c:stcv:12854549 === (OCoLC)253314165	Potential match
4. c:stcv:12854587 === BE-AnVE.c:stcv:12854587	But author and date didn't match
5. c:stcv:12854599 === BE-AnVE.c:stcv:12854599	But author and date didn't match
6. c:stcv:12857052 === BE-AnVE.c:stcv:12857052	But date didn't match
7. c:stcv:12857069 === BE-AnVE.c:stcv:12857069	Potential match
8. c:stcv:12857131 === BE-AnVE.c:stcv:12857131	But date didn't match
9. c:stcv:12857158 === (OCoLC)48731412	But author and date didn't match
10. c:stcv:12857275 === BE-AnVE.c:stcv:12857275	Potential match
11. c:stcv:12857374 === BE-AnVE.c:stcv:12857374	But author didn't match
12. c:stcv:12857912 === BE-AnVE.c:stcv:12902263	Potential match
13. c:stcv:12857923 === BE-AnVE.c:stcv:12857923	But author and date didn't match
14. c:stcv:12858124 === BE-AnVE.c:stcv:12858124	But date didn't match
15. c:stcv:128583

In [ ]:
# I used these urls to check my matches manually:
# hpb: https://kxp.k10plus.de/DB=1.77/SET=1/TTL=1/CMD?MATCFILTER=N&MATCSET=N&ACT0=&ACT=SRCHA&IKT=1016&SRT=YOP&ADI_IKT9200=&TRM=cid+BE-AnVE.c%3Astcv%3A12870121
# stcv: https://anet.be/brocade/brocade.phtml?UDdesk=uantwerpen&UDses=130246205%3A829800&UDstate=1&UDmode=&UDaccess=&UDrou=_EntryC%3Abopwexe&RDcgi=advancedsearchlinkexplorator&CDopSe=690538487&CDopSt=6&CDopLv=1&CDopOp=stcvopac
